# 0 - Information

# 1 - Packages

## 1.1 - Import of classical packages

In [1]:
# Math packages
import numpy as np

# Progress bar
from tqdm import tqdm

## 1.2 - Import of personal packages

In [2]:
# Import different tools functions
from Modules.Utils.Dropout import *
from Modules.Utils.ImportData import *
from Modules.Utils.Normalisation import *
from Modules.Utils.Preprocessing import *
from Modules.Utils.Predictions import *
from Modules.Utils.Transform import *

# Import functions for the data augmentation
from Modules.DataAugmentations.NoAugmentation import *
from Modules.DataAugmentations.ComplementarySequences import *
from Modules.DataAugmentations.PertubatedSequences import *

# Import functions for the embedding
from Modules.Embeddings.NoEmbedding import *
from Modules.Embeddings.SpectrumEmbedding import *
from Modules.Embeddings.DimismatchEmbedding import *
from Modules.Embeddings.MotifEmbedding import *
from Modules.Embeddings.WeightedDegreeEmbedding import *
from Modules.Embeddings.HotEncodingEmbedding import *
from Modules.Embeddings.FiguresEmbedding import *
from Modules.Embeddings.TraidEmbedding import *
# from Modules.Embeddings.HMMEmbedding import *

# Import functions for the selection of the model
from Modules.ModelSelection.CrossValidation import *
from Modules.ModelSelection.GridSearch import *

# Import functions for the kernels
from Modules.Kernels.LinearKernel import *
from Modules.Kernels.PolyKernel import *
from Modules.Kernels.DimismatchPolyKernel import *
from Modules.Kernels.CKN import *
from Modules.Kernels.GaussianKernel import *
from Modules.Kernels.SpectrumKernel import *
from Modules.Kernels.HMM import *

# Import function of model
from Modules.Models.KernelLogisticRegression import *
from Modules.Models.KernelSVM import *

# 2 - Data Import

In [3]:
# Extraction of the dataset
df_mat_dict = ImportData("./Data/Optionnal/", "./Data/", suffix="_mat100")
df_dict = ImportData("./Data/", "./Data/", header=0, sep=",")

# Display one of the datasets extracted (Xtr0_mat100)
display(df_dict[0][1].head())

,Id,seq
0,0,TTGACCGGAGGGATGAAAACAGTCTAAGACAGATTTATGGCGATAG...
1,1,TGTTGCTGGTGTGTAGAAACACTACTGATTTTTGTATCCAGCCACA...
2,2,TGTCTTAAACTTTACAGCTTTTTCATACAGTCTGTGTAGTACTAGA...
3,3,CAGGGAAAGGTTTGAAACGGAACGTAACAGCCCACGTTTTGCCAAA...
4,4,CCACCACCACTCCTGGCTAATTTTTGTATTTTTGGTAGAGACAGGG...


# 3 - Determining Best Model

In [65]:
a = np.arange(9, dtype=np.uint8).reshape((3, 3))
b = np.random.randint(0, 100, size=(3, 3), dtype=np.uint8)
result = np.zeros((3, 3))
np.dot(a, b, result)

ValueError: output array is not acceptable (must have the right datatype, number of dimensions, and be a C-Array)

In [63]:
import numpy as np
from numba import njit


def PolyKernel(X, Y, k=2, add_ones=False):
    """Compute the K matrix in the case of the linear kernel."""

    # Shape of X
    n, _ = np.shape(X)
    d, _ = np.shape(Y)
    

    # Convert X and Y
    X = np.array(X, dtype=np.float)
    Y = np.array(Y, dtype=np.float)

    @njit
    def subPolyKernel(X, Y):
        """Apply the dot product to X and Y."""

        return np.dot(X, Y.T)

    # Count the dot product
    result = subPolyKernel(X, Y)

    # Test if add ones
    if add_ones:
        # Compute results
        result = (np.array(result) + np.ones((n, d))) ** k

    else:
        # Compute results
        result = np.array(result) ** k

    return result


In [64]:
# Hyperparameters for DataAugmentation
hyperparameters_data_augmentation = {
                                        NoAugmentation: {},
#                                         PertubatedSequences: {"n": [2], "add_compl": [True, False]}
                                    }

# Hyperparameters for the embedding
hyperparameters_embedding = {
#                               NoEmbedding: {}
                              SpectrumEmbedding: {"d_l": [[5, 7, 12]]}
#                               FiguresEmbedding: {},
#                               DimismatchEmbedding: {"d": [[5, 6, 7]]}
                            }

# Hyperparameters of the kernels
hyperparameters_kernels = {
#     SpectrumKernel: {"d_l": [[5, 7, 12]]}
    PolyKernel: {
        "k": [2],
        "add_ones": [True]
    }
#     DimismatchPolyKernel: {
#         "m" : [3],
#         "k": [2],
#         "add_ones": [True],
#         "d_l": [[5, 6, 7]]
#     }
#    GaussianKernelBIS: {"sigma": [None, 10000, 10]}
}

# Hyper-parameters of the models
hyperparamters_models = {
                         KernelLogisticRegression: 
                             {"lamda": [10e-12], 
                              "informations": [False],
                              "preprocessing": [None],
                              "normalisation": [None],
                              "max_iter": [15],
                             },
#                          KernelSVM:
#                              {"lamda": [1, 0.01],
#                               "max_iter": [10e4],
#                               "tol": [10e-6],
#                               "informations": [False],
#                               "preprocessing": [Preprocessing, None]}
                        }



# GridSearch
[best_score, best_parameters_names,
best_parameters_values] = GridSearch(df_dict,
                                     hyperparameters_data_augmentation,
                                     hyperparameters_embedding,
                                     hyperparamters_models,
                                     hyperparameters_kernels,
                                     cv=5)

# Display result
print("Best Score: ", best_score)
print("Best Parameters: ", best_parameters_names)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/172281 [00:00<?, ?it/s]

  0%|          | 37/172281 [00:00<08:03, 356.56it/s]

  0%|          | 72/172281 [00:00<08:12, 349.78it/s]

  0%|          | 105/172281 [00:00<08:22, 342.93it/s]

  0%|          | 139/172281 [00:00<08:26, 340.07it/s]

  0%|          | 173/172281 [00:00<08:29, 337.82it/s]

  0%|          | 205/172281 [00:00<08:42, 329.28it/s]

  0%|          | 239/172281 [00:00<08:40, 330.65it/s]

  0%|          | 275/172281 [00:00<08:29, 337.35it/s]

  0%|          | 309/172281 [00:00<08:28, 338.03it/s]

  0%|          | 348/172281 [00:01<08:10, 350.20it/s]

  0%|          | 385/172281 [00:01<08:05, 354.26it/s]

  0%|          | 420/172281 [00:01<08:10, 350.73it/s]

  0%|          | 455/172281 [00:01<09:04, 315.36it/s]

  0%|          | 487/172281 [00:01<09:56, 288.08it/s]

  0%|          | 517/172281 [00:01<09:59, 286.65it/s]

  0%|          | 547/172281 [00:01<10:00, 286.00it/s]

  0%|          | 583/172281 [00:01<09:2

  6%|▌         | 10623/172281 [00:27<06:03, 444.85it/s]

  6%|▌         | 10668/172281 [00:27<06:07, 439.96it/s]

  6%|▌         | 10713/172281 [00:27<06:12, 434.21it/s]

  6%|▌         | 10757/172281 [00:27<06:11, 434.34it/s]

  6%|▋         | 10801/172281 [00:27<06:13, 431.94it/s]

  6%|▋         | 10845/172281 [00:27<06:12, 433.77it/s]

  6%|▋         | 10889/172281 [00:28<06:11, 434.56it/s]

  6%|▋         | 10934/172281 [00:28<06:08, 438.23it/s]

  6%|▋         | 10978/172281 [00:28<06:08, 437.82it/s]

  6%|▋         | 11022/172281 [00:28<06:08, 437.53it/s]

  6%|▋         | 11066/172281 [00:28<06:10, 435.56it/s]

  6%|▋         | 11110/172281 [00:28<06:09, 436.61it/s]

  6%|▋         | 11154/172281 [00:28<06:11, 434.24it/s]

  6%|▋         | 11198/172281 [00:28<06:10, 434.85it/s]

  7%|▋         | 11242/172281 [00:28<06:12, 432.88it/s]

  7%|▋         | 11286/172281 [00:29<06:12, 432.64it/s]

  7%|▋         | 11330/172281 [00:29<06:10, 434.78it/s]

  7%|▋         | 11374/172281 [

 13%|█▎        | 22108/172281 [00:54<05:43, 437.51it/s]

 13%|█▎        | 22152/172281 [00:54<05:43, 436.99it/s]

 13%|█▎        | 22197/172281 [00:54<05:42, 438.56it/s]

 13%|█▎        | 22242/172281 [00:54<05:40, 441.00it/s]

 13%|█▎        | 22287/172281 [00:54<05:39, 442.29it/s]

 13%|█▎        | 22332/172281 [00:54<05:39, 441.21it/s]

 13%|█▎        | 22377/172281 [00:54<05:41, 439.02it/s]

 13%|█▎        | 22421/172281 [00:54<05:42, 436.92it/s]

 13%|█▎        | 22465/172281 [00:54<05:42, 437.38it/s]

 13%|█▎        | 22510/172281 [00:54<05:40, 439.80it/s]

 13%|█▎        | 22554/172281 [00:55<05:41, 438.98it/s]

 13%|█▎        | 22599/172281 [00:55<05:39, 440.56it/s]

 13%|█▎        | 22644/172281 [00:55<05:38, 441.58it/s]

 13%|█▎        | 22689/172281 [00:55<05:41, 438.42it/s]

 13%|█▎        | 22733/172281 [00:55<05:44, 434.54it/s]

 13%|█▎        | 22777/172281 [00:55<05:44, 433.36it/s]

 13%|█▎        | 22821/172281 [00:55<05:47, 430.70it/s]

 13%|█▎        | 22865/172281 [

 20%|█▉        | 33663/172281 [01:20<05:16, 437.88it/s]

 20%|█▉        | 33707/172281 [01:20<05:18, 435.66it/s]

 20%|█▉        | 33751/172281 [01:20<05:17, 435.87it/s]

 20%|█▉        | 33795/172281 [01:20<05:18, 435.13it/s]

 20%|█▉        | 33840/172281 [01:20<05:15, 439.04it/s]

 20%|█▉        | 33885/172281 [01:21<05:13, 441.59it/s]

 20%|█▉        | 33930/172281 [01:21<05:14, 439.72it/s]

 20%|█▉        | 33974/172281 [01:21<05:15, 438.31it/s]

 20%|█▉        | 34018/172281 [01:21<05:17, 435.63it/s]

 20%|█▉        | 34063/172281 [01:21<05:15, 438.24it/s]

 20%|█▉        | 34108/172281 [01:21<05:13, 440.70it/s]

 20%|█▉        | 34153/172281 [01:21<05:15, 437.80it/s]

 20%|█▉        | 34197/172281 [01:21<05:19, 431.96it/s]

 20%|█▉        | 34241/172281 [01:21<05:19, 431.84it/s]

 20%|█▉        | 34286/172281 [01:21<05:16, 436.29it/s]

 20%|█▉        | 34331/172281 [01:22<05:13, 439.90it/s]

 20%|█▉        | 34376/172281 [01:22<05:12, 441.96it/s]

 20%|█▉        | 34421/172281 [

 26%|██▌       | 44937/172281 [01:47<05:15, 403.46it/s]

 26%|██▌       | 44978/172281 [01:47<05:34, 380.89it/s]

 26%|██▌       | 45017/172281 [01:47<05:45, 368.31it/s]

 26%|██▌       | 45058/172281 [01:47<05:35, 378.99it/s]

 26%|██▌       | 45103/172281 [01:47<05:20, 397.05it/s]

 26%|██▌       | 45148/172281 [01:48<05:09, 410.11it/s]

 26%|██▌       | 45191/172281 [01:48<05:05, 415.37it/s]

 26%|██▋       | 45233/172281 [01:48<05:06, 414.90it/s]

 26%|██▋       | 45275/172281 [01:48<05:07, 413.44it/s]

 26%|██▋       | 45318/172281 [01:48<05:04, 417.22it/s]

 26%|██▋       | 45363/172281 [01:48<04:58, 424.96it/s]

 26%|██▋       | 45406/172281 [01:48<05:16, 400.54it/s]

 26%|██▋       | 45450/172281 [01:48<05:08, 411.26it/s]

 26%|██▋       | 45492/172281 [01:48<05:06, 413.49it/s]

 26%|██▋       | 45537/172281 [01:49<05:00, 422.31it/s]

 26%|██▋       | 45580/172281 [01:49<05:09, 408.73it/s]

 26%|██▋       | 45623/172281 [01:49<05:05, 414.44it/s]

 27%|██▋       | 45667/172281 [

...

 96%|█████████▋| 165947/172281 [06:41<00:14, 435.12it/s]

 96%|█████████▋| 165991/172281 [06:41<00:14, 433.87it/s]

 96%|█████████▋| 166035/172281 [06:41<00:14, 431.38it/s]

 96%|█████████▋| 166079/172281 [06:41<00:14, 429.97it/s]

 96%|█████████▋| 166123/172281 [06:41<00:14, 432.47it/s]

 96%|█████████▋| 166167/172281 [06:42<00:14, 434.39it/s]

 96%|█████████▋| 166211/172281 [06:42<00:13, 435.78it/s]

 97%|█████████▋| 166255/172281 [06:42<00:13, 436.29it/s]

 97%|█████████▋| 166299/172281 [06:42<00:13, 436.14it/s]

 97%|█████████▋| 166343/172281 [06:42<00:13, 435.53it/s]

 97%|█████████▋| 166387/172281 [06:42<00:13, 428.37it/s]

 97%|█████████▋| 166430/172281 [06:42<00:13, 427.03it/s]

 97%|█████████▋| 166474/172281 [06:42<00:13, 428.35it/s]

 97%|█████████▋| 166518/172281 [06:42<00:13, 430.63it/s]

 97%|█████████▋| 166562/172281 [06:43<00:13, 427.16it/s]

 97%|█████████▋| 166606/172281 [06:43<00:13, 429.33it/s]

 97%|█████████▋| 166650/172281 [06:43<00:13, 429.85it/s]

 97%|█████████

  3%|▎         | 4775/169849 [00:12<06:46, 405.93it/s]

  3%|▎         | 4817/169849 [00:12<06:43, 409.45it/s]

  3%|▎         | 4860/169849 [00:12<06:37, 414.69it/s]

  3%|▎         | 4903/169849 [00:12<06:34, 417.88it/s]

  3%|▎         | 4948/169849 [00:12<06:28, 424.35it/s]

  3%|▎         | 4992/169849 [00:12<06:26, 426.80it/s]

  3%|▎         | 5035/169849 [00:12<06:26, 426.79it/s]

  3%|▎         | 5079/169849 [00:12<06:25, 427.93it/s]

  3%|▎         | 5123/169849 [00:12<06:23, 429.92it/s]

  3%|▎         | 5167/169849 [00:12<06:23, 429.75it/s]

  3%|▎         | 5210/169849 [00:13<06:23, 429.27it/s]

  3%|▎         | 5253/169849 [00:13<06:29, 423.02it/s]

  3%|▎         | 5296/169849 [00:13<06:31, 420.78it/s]

  3%|▎         | 5339/169849 [00:13<06:31, 419.90it/s]

  3%|▎         | 5382/169849 [00:13<06:39, 411.79it/s]

  3%|▎         | 5424/169849 [00:13<07:10, 382.37it/s]

  3%|▎         | 5463/169849 [00:13<07:10, 382.19it/s]

  3%|▎         | 5505/169849 [00:13<07:00, 391.1

  9%|▉         | 16043/169849 [00:39<05:59, 427.72it/s]

  9%|▉         | 16087/169849 [00:39<05:57, 430.45it/s]

  9%|▉         | 16131/169849 [00:39<05:58, 428.51it/s]

 10%|▉         | 16175/169849 [00:39<05:57, 430.42it/s]

 10%|▉         | 16219/169849 [00:39<05:57, 430.16it/s]

 10%|▉         | 16263/169849 [00:39<05:56, 430.29it/s]

 10%|▉         | 16307/169849 [00:39<05:57, 429.60it/s]

 10%|▉         | 16350/169849 [00:39<05:59, 427.57it/s]

 10%|▉         | 16393/169849 [00:39<05:59, 426.78it/s]

 10%|▉         | 16436/169849 [00:39<06:03, 422.49it/s]

 10%|▉         | 16479/169849 [00:40<06:04, 420.69it/s]

 10%|▉         | 16522/169849 [00:40<06:04, 420.54it/s]

 10%|▉         | 16565/169849 [00:40<06:07, 417.42it/s]

 10%|▉         | 16609/169849 [00:40<06:02, 422.53it/s]

 10%|▉         | 16653/169849 [00:40<05:59, 426.43it/s]

 10%|▉         | 16696/169849 [00:40<05:58, 426.86it/s]

 10%|▉         | 16739/169849 [00:40<06:11, 412.38it/s]

 10%|▉         | 16783/169849 [

 16%|█▌        | 27316/169849 [01:05<05:24, 438.63it/s]

 16%|█▌        | 27361/169849 [01:06<05:24, 439.62it/s]

 16%|█▌        | 27405/169849 [01:06<05:24, 439.49it/s]

 16%|█▌        | 27449/169849 [01:06<05:24, 438.56it/s]

 16%|█▌        | 27493/169849 [01:06<05:28, 433.85it/s]

 16%|█▌        | 27537/169849 [01:06<05:28, 433.21it/s]

 16%|█▌        | 27581/169849 [01:06<05:27, 434.59it/s]

 16%|█▋        | 27625/169849 [01:06<05:26, 435.52it/s]

 16%|█▋        | 27669/169849 [01:06<05:27, 434.27it/s]

 16%|█▋        | 27714/169849 [01:06<05:25, 436.24it/s]

 16%|█▋        | 27760/169849 [01:06<05:22, 441.06it/s]

 16%|█▋        | 27805/169849 [01:07<05:24, 438.23it/s]

 16%|█▋        | 27849/169849 [01:07<05:25, 436.72it/s]

 16%|█▋        | 27893/169849 [01:07<05:26, 434.72it/s]

 16%|█▋        | 27937/169849 [01:07<05:27, 433.32it/s]

 16%|█▋        | 27981/169849 [01:07<05:26, 434.66it/s]

 16%|█▋        | 28025/169849 [01:07<05:25, 435.21it/s]

 17%|█▋        | 28069/169849 [

KeyboardInterrupt: 

# 4 - Computation of the Predictions for the Best Parameters

## 4.1 - Computation of the best parameters

In [23]:
# Definition of the data augmentation function
data_aug = DataAugmentationDefault(NoAugmentation, {})


# Defintion of the embedding
embedding = EmbeddingDefault(SpectrumEmbedding , {"d_l": [5, 7, 12]})
# 

# Definition of the kernel
kernel = KernelDefault(PolyKernel, {"k": 2})

# Definition of the model
model = KernelLogisticRegression(kernel, informations=True,  lamda=10, max_iter=15,
                                 preprocessing=None)

# Defintion of best parameters values
best_parameters_values = {"Data Augmentation": {"Function": data_aug},
                          "Embedding": {"Function": embedding},
                          "Kernel": {"Function": kernel},
                          "Model": {"Function": model}}

In [24]:
# Computation of the predicition
predictions = Prediction(best_parameters_values, df_dict)

# Display predicitons
predictions

100%|██████████| 3/3 [00:00<00:00, 22.46it/s]


KeyboardInterrupt: 

In [ ]:
np.mean(predictions, axis=0)

## 4.2 - Save predicitons into a csv

In [ ]:
np.savetxt("./Resultats/Predictions_Test_Spectrum.csv", predictions, 
           fmt='%i', delimiter=",", header="Id,Bound", comments='')

# Testing

## 1 - Study the result

In [4]:
df = pd.read_csv("./Resultats/grid_search_res.csv",  sep='\t')

# Display df
df.sort_values("score", inplace=True, ascending=False)
df

,Unnamed: 0,data_aug_hp,data_aug_type,embedding_hp,embedding_type,kernel_hp,kernel_type,model_hp,model_type,score,scores_1,scores_1_mean,scores_2,scores_2_mean,scores_3,scores_3_mean
0,0,{},ComplementarySequences,"{'d_l': [5, 10, 15]}",SpectrumEmbedding,"{'k': 2, 'add_ones': False}",PolyKernel,"{'lamda': 1, 'preprocessing': None, 'informati...",KernelLogisticRegression,0.205000,"[0.59, 0.6225, 0.6225, 0.625, 0.615]",0.6150,"[0, 0, 0, 0, 0]",0.0,"[0, 0, 0, 0, 0]",0.0
12,12,{},ComplementarySequences,"{'d_l': [5, 10, 15]}",SpectrumEmbedding,"{'k': 2, 'add_ones': False}",PolyKernel,"{'lamda': 0.0001, 'preprocessing': None, 'info...",KernelLogisticRegression,0.204333,"[0.5925, 0.6175, 0.62, 0.62, 0.615]",0.6130,"[0, 0, 0, 0, 0]",0.0,"[0, 0, 0, 0, 0]",0.0
7,7,{},ComplementarySequences,"{'d_l': [5, 10, 15]}",SpectrumEmbedding,"{'k': 2, 'add_ones': False}",PolyKernel,"{'lamda': 0.01, 'preprocessing': None, 'inform...",KernelLogisticRegression,0.204167,"[0.59, 0.615, 0.62, 0.62, 0.6175]",0.6125,"[0, 0, 0, 0, 0]",0.0,"[0, 0, 0, 0, 0]",0.0
13,13,{},ComplementarySequences,"{'d_l': [5, 7, 12]}",SpectrumEmbedding,"{'k': 2, 'add_ones': False}",PolyKernel,"{'lamda': 1, 'preprocessing': None, 'informati...",KernelLogisticRegression,0.203500,"[0.5975, 0.615, 0.625, 0.6125, 0.6025]",0.6105,"[0, 0, 0, 0, 0]",0.0,"[0, 0, 0, 0, 0]",0.0
6,6,{},ComplementarySequences,"{'d_l': [5, 7, 12]}",SpectrumEmbedding,"{'k': 2, 'add_ones': False}",PolyKernel,"{'lamda': 0.0001, 'preprocessing': None, 'info...",KernelLogisticRegression,0.203000,"[0.595, 0.615, 0.6175, 0.615, 0.6025]",0.6090,"[0, 0, 0, 0, 0]",0.0,"[0, 0, 0, 0, 0]",0.0
3,3,{},ComplementarySequences,"{'d_l': [5, 7, 12]}",SpectrumEmbedding,"{'k': 2, 'add_ones': False}",PolyKernel,"{'lamda': 0.01, 'preprocessing': None, 'inform...",KernelLogisticRegression,0.202500,"[0.5975, 0.6125, 0.615, 0.61, 0.6025]",0.6075,"[0, 0, 0, 0, 0]",0.0,"[0, 0, 0, 0, 0]",0.0
8,8,{},ComplementarySequences,"{'d_l': [5, 7, 12]}",SpectrumEmbedding,"{'k': 2, 'add_ones': False}",PolyKernel,"{'lamda': 100, 'preprocessing': None, 'informa...",KernelLogisticRegression,0.197500,"[0.5375, 0.6025, 0.6575, 0.5925, 0.5725]",0.5925,"[0, 0, 0, 0, 0]",0.0,"[0, 0, 0, 0, 0]",0.0
9,9,{},ComplementarySequences,"{'d_l': [5, 10, 15]}",SpectrumEmbedding,"{'k': 2, 'add_ones': False}",PolyKernel,"{'lamda': 100, 'preprocessing': None, 'informa...",KernelLogisticRegression,0.195333,"[0.5275, 0.61, 0.65, 0.59, 0.5525]",0.5860,"[0, 0, 0, 0, 0]",0.0,"[0, 0, 0, 0, 0]",0.0
4,4,{},ComplementarySequences,"{'d_l': [5, 7, 12]}",SpectrumEmbedding,"{'k': 2, 'add_ones': False}",PolyKernel,"{'lamda': 1000000000.0, 'preprocessing': None,...",KernelLogisticRegression,0.188167,"[0.51, 0.5925, 0.59, 0.5625, 0.5675]",0.5645,"[0, 0, 0, 0, 0]",0.0,"[0, 0, 0, 0, 0]",0.0
5,5,{},ComplementarySequences,"{'d_l': [5, 7, 12]}",SpectrumEmbedding,"{'k': 2, 'add_ones': False}",PolyKernel,"{'lamda': 1000000.0, 'preprocessing': None, 'i...",KernelLogisticRegression,0.188167,"[0.51, 0.5925, 0.59, 0.5625, 0.5675]",0.5645,"[0, 0, 0, 0, 0]",0.0,"[0, 0, 0, 0, 0]",0.0


In [ ]:
# from sklearn.feature_extraction.text import TfidfTransformer

# counts = spectrumEmbedding(df_dict[0][0], len_sq=4)
# new_X_train = transformer.fit_transform(counts).toarray()

# # transformer = TfidfTransformer(smooth_idf=False) # True

# for i in [5, 10]:
    
#     counts = spectrumEmbedding(df_dict[0][0], len_sq=i)
#     new_X_train = transformer.fit_transform(counts).toarray()

#     model = SVC(gamma="scale", C=100)
    
#     # Execute a cross validation on the model
#     gdm = GridSearchCV(model, hp, scoring="accuracy", cv=5)

#     # Fit the model and find the best parameters and score
#     gdm.fit(new_X_train, y_train)
    
#     print("Iteration {}, score {}".format(i, gdm.best_score_))

# 888 - CKN